<a href="https://colab.research.google.com/github/Greencapral/Python_Courses/blob/main/%D0%97%D0%B0%D0%BD%D1%8F%D1%82%D0%B8%D0%B5_13_%D0%A2%D0%B5%D1%81%D1%82%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1. Введение**  

---

### **1.1. Цели занятия**  

Тестирование — это важная часть разработки, которая позволяет находить ошибки на ранних этапах и предотвращать появление новых багов при изменении кода. На этом занятии мы научимся писать тесты с помощью библиотеки `unittest`, разберёмся, как запускать тесты автоматически, и познакомимся с инструментами для подмены зависимостей.

Основные задачи, которые мы решим:
- Освоим работу с `unittest`, научимся создавать тестовые классы и тестовые методы.
- Разберём, какие проверки можно выполнять с помощью `assert`-методов.
- Изучим фикстуры для подготовки тестовой среды.
- Познакомимся с параметризацией тестов, чтобы проверять код на разных входных данных.
- Рассмотрим `unittest.mock` для подмены зависимостей.

---

### **1.2. Краткое содержание**  

В рамках занятия мы:
1. Разберём основные принципы тестирования и зачем они нужны.
2. Научимся создавать тесты в `unittest` и использовать его основные методы.
3. Освоим фикстуры, которые позволяют выполнять подготовку перед тестами и очистку после них.
4. Разберёмся с параметризацией тестов, чтобы избежать дублирования кода.
5. Познакомимся с `unittest.mock` для подмены зависимостей и тестирования сложных сценариев.

Тестирование помогает автоматизировать процесс проверки кода, снижает количество ошибок в продакшне и упрощает рефакторинг.

**Примечание**: *Поскольку код данного занятия включает тестирование в отдельных модулях, код в рамках данного документа **не исполняемый** однако, вы можете повторить структуру локально и протестировать его на своем компьютере*

## **2. Основы тестирования с `unittest`**  

Тестирование помогает проверять работоспособность кода автоматически, избегая ошибок при изменениях. В Python для этого используется встроенный модуль `unittest`. Он позволяет писать тесты, группировать их в классы, использовать вспомогательные методы (`assertEqual`, `assertRaises` и др.) и запускать тесты через командную строку.

---

### **2.1. Структура проекта с тестами**  

При организации тестов в реальном проекте их обычно хранят в отдельной директории `tests`. Это позволяет отделить тестовый код от основной логики.

Пример структуры проекта:

```plaintext
project/
├── main.py                # Основной код приложения
├── utils.py               # Вспомогательные функции
├── tests/                 # Папка с тестами
│   ├── __init__.py        # Позволяет воспринимать папку как пакет
│   ├── test_utils.py      # Тесты для utils.py
│   └── test_main.py       # Тесты для main.py
```

Тесты принято размещать в отдельных файлах, а имена тестовых файлов должны начинаться с `test_`, чтобы тестовый раннер мог их найти автоматически.

---

### **2.2. Создание первого теста**

Допустим, у нас есть модуль `utils.py` с функцией сложения:

```python
# utils.py
def add(a, b):
    return a + b
```

Теперь создадим тест для этой функции.

Создадим файл `tests/test_utils.py`:

```python
# tests/test_utils.py
import unittest
from utils import add  # Импортируем тестируемую функцию

class TestUtils(unittest.TestCase):

    def test_add(self):
        """Проверка сложения двух чисел"""
        self.assertEqual(add(2, 3), 5)
        self.assertEqual(add(-1, 1), 0)
        self.assertEqual(add(0, 0), 0)

if __name__ == "__main__":
    unittest.main()
```

---

### **2.3. Запуск тестов**

Тесты можно запустить командой:

```bash
python -m unittest discover tests
```

Или выполнить конкретный файл с тестами:

```bash
python -m unittest tests.test_utils
```

---

### **2.4. Что выйдет в итоге?**

Если все тесты пройдут успешно, в консоли появится вывод:

```plaintext
...
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK
```

Если один из тестов не пройдёт (например, изменим `add(2, 3)` на `6`), увидим ошибку:

```plaintext
======================================================================
FAIL: test_add (tests.test_utils.TestUtils)
Проверка сложения двух чисел
----------------------------------------------------------------------
Traceback (most recent call last):
  File "tests/test_utils.py", line 9, in test_add
    self.assertEqual(add(2, 3), 6)
AssertionError: 5 != 6

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (failures=1)
```

---

### **2.5. Основные методы `unittest.TestCase`**

В тестах используются специальные методы для проверки значений:

- `assertEqual(a, b)` – проверяет, что `a == b`.
- `assertNotEqual(a, b)` – проверяет, что `a != b`.
- `assertTrue(x)` – проверяет, что `x == True`.
- `assertFalse(x)` – проверяет, что `x == False`.
- `assertRaises(Exception, func, *args)` – проверяет, что при вызове `func(*args)` выбрасывается исключение.

Пример проверки деления на ноль:

```python
# tests/test_utils.py
import unittest
from utils import add

class TestUtils(unittest.TestCase):

    def test_add(self):
        self.assertEqual(add(2, 3), 5)

    def test_divide_by_zero(self):
        with self.assertRaises(ZeroDivisionError):
            1 / 0  # Проверяем, что деление на 0 вызывает ошибку

if __name__ == "__main__":
    unittest.main()
```

---

## **3. Фикстуры в `unittest`**  

Фикстуры в тестировании используются для подготовки тестовой среды перед выполнением тестов и её очистки после. В `unittest` фикстуры реализуются с помощью методов `setUp()`, `tearDown()`, `setUpClass()` и `tearDownClass()`.

---

### **3.1. Что такое фикстуры и зачем они нужны**  

При тестировании часто требуется подготовить данные перед запуском тестов (например, создать временные файлы, подключиться к базе данных или создать объект для тестирования). После тестов нужно очистить ресурсы. Фикстуры автоматизируют эти процессы.

---

### **3.2. Использование `setUp()` и `tearDown()`**  

Метод `setUp()` выполняется перед **каждым тестом**, а `tearDown()` — после каждого теста. Это удобно, когда тесты требуют одинаковой подготовки.

#### **Пример: тестируем класс `BankAccount`**  

```python
# bank.py
class BankAccount:
    def __init__(self, owner, balance=0):
        self.owner = owner
        self.balance = balance

    def deposit(self, amount):
        self.balance += amount

    def withdraw(self, amount):
        if amount > self.balance:
            raise ValueError("Недостаточно средств")
        self.balance -= amount
```

Теперь создадим тесты с фикстурами.

```python
# tests/test_bank.py
import unittest
from bank import BankAccount

class TestBankAccount(unittest.TestCase):

    def setUp(self):
        """Подготовка тестовых данных перед каждым тестом"""
        self.account = BankAccount("Алиса", 1000)

    def tearDown(self):
        """Очистка после теста (если требуется)"""
        del self.account

    def test_deposit(self):
        """Проверяем пополнение счёта"""
        self.account.deposit(500)
        self.assertEqual(self.account.balance, 1500)

    def test_withdraw(self):
        """Проверяем снятие средств"""
        self.account.withdraw(200)
        self.assertEqual(self.account.balance, 800)

    def test_withdraw_not_enough_funds(self):
        """Проверяем ошибку при недостатке средств"""
        with self.assertRaises(ValueError):
            self.account.withdraw(2000)

if __name__ == "__main__":
    unittest.main()
```

**Как это работает?**
1. **Перед каждым тестом** создаётся новый объект `BankAccount` с балансом 1000.
2. Тест использует этот объект и проверяет методы `deposit()` и `withdraw()`.
3. После теста объект удаляется в `tearDown()`.

Запуск тестов:
```bash
python -m unittest tests/test_bank.py
```

Если все тесты успешны:
```plaintext
...
----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK
```

Если будет ошибка:
```plaintext
======================================================================
FAIL: test_withdraw (tests.test_bank.TestBankAccount)
Проверяем снятие средств
----------------------------------------------------------------------
Traceback (most recent call last):
  File "tests/test_bank.py", line 20, in test_withdraw
    self.assertEqual(self.account.balance, 900)
AssertionError: 800 != 900

----------------------------------------------------------------------
Ran 3 tests in 0.002s

FAILED (failures=1)
```

---

### **3.3. Использование `setUpClass()` и `tearDownClass()`**  

Эти методы выполняются **один раз для всего тестового класса**, а не перед каждым тестом.

Пример: подключение к базе данных перед тестами.

```python
import unittest

class TestDatabase(unittest.TestCase):

    @classmethod
    def setUpClass(cls):
        """Выполняется перед всеми тестами в классе"""
        print("Подключение к тестовой базе данных")

    @classmethod
    def tearDownClass(cls):
        """Выполняется после всех тестов в классе"""
        print("Закрытие соединения с базой данных")

    def test_query_1(self):
        print("Выполнение теста 1")

    def test_query_2(self):
        print("Выполнение теста 2")

if __name__ == "__main__":
    unittest.main()
```

При запуске тестов:
```plaintext
Подключение к тестовой базе данных
Выполнение теста 1
Выполнение теста 2
Закрытие соединения с базой данных
```

`setUpClass()` полезен, когда тестам нужно **общее подключение**, например, к базе данных, а `setUp()` — если требуется изолированная настройка перед каждым тестом.

---

### **Вывод**  
- `setUp()` и `tearDown()` запускаются перед и после **каждого** теста.
- `setUpClass()` и `tearDownClass()` запускаются **один раз** на весь тестовый класс.
- Фикстуры помогают автоматизировать подготовку и очистку тестового окружения.


## **4. Параметризация тестов**  

Часто при тестировании одной функции требуется проверить её на разных входных данных. Вместо того чтобы дублировать код тестов, можно использовать параметризацию. В `unittest` это можно сделать с помощью `subTest()` или сторонней библиотеки `ddt`.

---

### **4.1. Как передавать разные входные данные в тесты**

Допустим, у нас есть функция вычисления площади прямоугольника:

```python
# geometry.py
def rectangle_area(width, height):
    if width <= 0 or height <= 0:
        raise ValueError("Ширина и высота должны быть положительными числами")
    return width * height
```

Теперь создадим параметризованный тест.

---

### **4.2. Использование `subTest()`**

Метод `subTest()` позволяет запускать один и тот же тест с разными входными данными.

```python
# tests/test_geometry.py
import unittest
from geometry import rectangle_area

class TestRectangleArea(unittest.TestCase):

    def test_rectangle_area(self):
        """Проверка вычисления площади с разными параметрами"""
        test_cases = [
            (3, 4, 12),
            (5, 10, 50),
            (2, 7, 14)
        ]
        for width, height, expected in test_cases:
            with self.subTest(width=width, height=height):
                self.assertEqual(rectangle_area(width, height), expected)

    def test_rectangle_area_invalid_values(self):
        """Проверка исключения при некорректных значениях"""
        with self.assertRaises(ValueError):
            rectangle_area(-1, 5)
        with self.assertRaises(ValueError):
            rectangle_area(5, 0)

if __name__ == "__main__":
    unittest.main()
```

Запуск тестов:
```bash
python -m unittest tests/test_geometry.py
```

Если все тесты успешны:
```plaintext
...
----------------------------------------------------------------------
Ran 2 tests in 0.001s

OK
```

Если один из тестов не пройдёт, `subTest()` покажет, какие параметры привели к ошибке.

```plaintext
======================================================================
FAIL: test_rectangle_area (__main__.TestRectangleArea) (width=3, height=4)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "tests/test_geometry.py", line 12, in test_rectangle_area
    self.assertEqual(rectangle_area(width, height), expected)
AssertionError: 11 != 12

----------------------------------------------------------------------
Ran 2 tests in 0.002s

FAILED (failures=1)
```

---

### **4.3. Использование `ddt` для параметризации тестов**

Библиотека `ddt` (`Data-Driven Testing`) позволяет проще передавать параметры в тесты.

**Установка `ddt`** (если ещё не установлена):
```bash
pip install ddt
```

**Пример тестов с `ddt`:**
```python
# tests/test_geometry.py
import unittest
from ddt import ddt, data, unpack
from geometry import rectangle_area

@ddt
class TestRectangleArea(unittest.TestCase):

    @data((3, 4, 12), (5, 10, 50), (2, 7, 14))
    @unpack
    def test_rectangle_area(self, width, height, expected):
        """Проверка вычисления площади с разными параметрами"""
        self.assertEqual(rectangle_area(width, height), expected)

if __name__ == "__main__":
    unittest.main()
```

**Разбор кода:**
- `@ddt` — помечает класс как параметризованный.
- `@data(...)` — передаёт список кортежей с входными данными.
- `@unpack` — распаковывает кортежи в аргументы тестовой функции.

Вывод тестов будет аналогичным `subTest()`, но код становится компактнее.

---

### **Вывод**
- `subTest()` позволяет тестировать разные данные в одном тесте.
- `ddt` упрощает параметризацию, особенно при работе с большими наборами тестовых данных.
- Параметризованные тесты позволяют избежать дублирования кода и сделать тестирование более гибким.

## **5. Модуль `unittest.mock`**  

При тестировании программ иногда необходимо изолировать код от зависимостей, например:  
- **Отключить реальные запросы к API**.  
- **Подменить взаимодействие с базой данных**.  
- **Имитировать работу системных функций**.  

Для этого в `unittest` есть `mock` — инструмент для подмены зависимостей.

---

### **5.1. Для чего нужен `mock`**  

`unittest.mock` позволяет:
- Подменять объекты, которые не должны выполняться во время тестов.
- Проверять, были ли вызваны функции и с какими аргументами.
- Эмулировать поведение реальных объектов.

Пример ситуаций, где `mock` полезен:
- Вызовы API (`requests.get()`).
- Работа с файлами (`open()`).
- Взаимодействие с базой данных.

---

### **5.2. Основные методы `unittest.mock`**  

- `Mock()` — создаёт заглушку объекта.
- `patch()` — временно заменяет объект.
- `assert_called_once()` — проверяет, что функция была вызвана один раз.
- `assert_called_with(*args, **kwargs)` — проверяет аргументы вызова.

---

### **5.3. Примеры использования моков**  

#### **1. Подмена API-запроса с `patch()`**  

Допустим, у нас есть код, который получает данные с сервера:

```python
# api.py
import requests

def get_weather(city):
    url = f"https://weather-api.com/{city}"
    response = requests.get(url)
    return response.json()
```

Чтобы тестировать `get_weather()`, мы заменим `requests.get()` мок-объектом.

**Тест с `mock.patch()`**:

```python
# tests/test_api.py
import unittest
from unittest.mock import patch
from api import get_weather

class TestWeatherAPI(unittest.TestCase):

    @patch("api.requests.get")  # Подменяем `requests.get`
    def test_get_weather(self, mock_get):
        """Проверяем работу API с мок-ответом"""
        mock_get.return_value.json.return_value = {"temperature": 20}

        result = get_weather("Moscow")
        
        self.assertEqual(result, {"temperature": 20})
        mock_get.assert_called_once_with("https://weather-api.com/Moscow")

if __name__ == "__main__":
    unittest.main()
```

**Как это работает?**  
- `@patch("api.requests.get")` заменяет `requests.get()` заглушкой.
- `mock_get.return_value.json.return_value = {"temperature": 20}` имитирует ответ API.
- `mock_get.assert_called_once_with(...)` проверяет, что URL вызван правильно.

---

#### **2. Подмена системных вызовов (`open()`)**  

Допустим, у нас есть функция, которая читает содержимое файла:

```python
# file_reader.py
def read_file(filename):
    with open(filename, "r") as f:
        return f.read()
```

Вместо реального чтения файла подменим `open()`:

```python
# tests/test_file_reader.py
import unittest
from unittest.mock import mock_open, patch
from file_reader import read_file

class TestFileReader(unittest.TestCase):

    @patch("builtins.open", new_callable=mock_open, read_data="Mock data")
    def test_read_file(self, mock_file):
        """Проверяем чтение файла с мок-данными"""
        result = read_file("fake_file.txt")
        
        self.assertEqual(result, "Mock data")
        mock_file.assert_called_once_with("fake_file.txt", "r")

if __name__ == "__main__":
    unittest.main()
```

**Разбор кода:**
- `mock_open(read_data="Mock data")` создаёт заглушку для `open()`.
- `@patch("builtins.open", new_callable=mock_open, read_data="Mock data")` подменяет `open()`.
- `mock_file.assert_called_once_with("fake_file.txt", "r")` проверяет корректность вызова.

---

### **Вывод**  
- `unittest.mock` позволяет изолировать код от внешних зависимостей.  
- `patch()` заменяет объекты на моки, а `Mock()` создаёт заглушки.  
- Это полезно при тестировании API, работы с файлами и системными вызовами.  



## 6. Задания для самостоятельной практики

Поскольку на данном занятии и так предусмотрено домашнее занятие, предлагаем вам попрактиковаться именно при его выполнении, однако, советуем повторить код занятия на своем компьютере и попрактиковаться в тестировании, чтобы посмотреть, как работает библиотека.

## **7. Заключение**  

Тестирование — это не просто проверка кода, а важный инструмент для повышения надёжности программ. Сегодня мы познакомились с `unittest` и `unittest.mock`, разобрали, как писать тесты, автоматизировать их выполнение и изолировать код от внешних зависимостей.  

Мы начали с **основ тестирования**, создав первые тесты и запустив их. Затем разобрали **фикстуры**, которые помогают подготавливать данные перед тестами и очищать их после. Изучили **параметризацию**, которая позволяет тестировать один и тот же код с разными входными данными. Наконец, мы разобрались с **моками**, которые позволяют тестировать код без реального выполнения запросов к API, работе с файлами или базами данных.  

Тесты помогают избежать неожиданных ошибок и ускоряют разработку. Чем раньше они пишутся, тем легче поддерживать код и добавлять в него новые функции. Главное — делать тестирование регулярной практикой, а не чем-то, что откладывается «на потом».